In [2]:
import sys
!{sys.executable} -m pip install matplotlib seaborn


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------- ------------------------- 2.9/8.1 MB 18.2 MB/s eta 0:00:01
   --------------------------- ------------ 5.5/8.1 MB 14.9 MB/s eta 0:00:01
   -------------------------------------- - 7.9/8.1 MB 13.8 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------- ---------------- 1.6/2.7 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 6.4 MB/s eta 0:00:00

   ----- ---------------------------------- 1/8 [pillow]
   ----- ---------------------------------- 1/8 [pillow]
   --------------- ------------------------ 3/8 [fonttools]
   --------------- ------------------------ 3/8 [fonttools]
   --------------- --

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# --- Tambahkan sel baru di atas evaluasi ---

import pandas as pd
from pathlib import Path

BASE = Path(r"D:/SEMESTER 6/PROJECT CBR")
EVAL = BASE / "data/eval"
RES = BASE / "data/results"
EVAL.mkdir(parents=True, exist_ok=True)
RES.mkdir(parents=True, exist_ok=True)

# Dummy retrieval_eval_triple.csv
df_ret = pd.DataFrame({
    "model": ["TF-IDF", "SVM", "BERT", "TF-IDF", "SVM", "BERT"],
    "precision@5": [0.8, 0.7, 0.9, 0.75, 0.72, 0.88],
    "recall@5": [0.6, 0.65, 0.85, 0.62, 0.68, 0.83],
    "f1@5": [0.68, 0.67, 0.87, 0.69, 0.7, 0.85],
    "accuracy@5": [0.7, 0.66, 0.88, 0.72, 0.69, 0.86]
})
df_ret.to_csv(EVAL / "retrieval_eval_triple.csv", index=False)

# Dummy predictions.csv
df_pred = pd.DataFrame({
    "query_id": [1, 2, 3, 4, 5],
    "predicted_solution": [
        "Dipidana 2 tahun",
        "Mengembalikan uang koperasi",
        "Dijatuhi pidana penjara 5 tahun",
        "Gugatan tidak diterima",
        "Dihukum 1 tahun 6 bulan"
    ],
    "method": ["TF-IDF", "SVM", "BERT", "TF-IDF", "SVM"]
})
df_pred.to_csv(RES / "predictions.csv", index=False)

In [8]:
# I. Evaluasi Retrieval (TF-IDF, SVM, BERT)
# ─────────────────────────────────────────────
df_ret = pd.read_csv(retrieval_fp)

# Hitung rata-rata per model
mean_metrics = df_ret.groupby("model")[["precision@5", "recall@5", "f1@5", "accuracy@5"]].mean().round(3)

In [9]:
# Simpan ulang metrik ke CSV
mean_metrics.to_csv(EVAL / "retrieval_metrics.csv")
print("📊 Rata-rata Metrik Retrieval:")
print(mean_metrics)

# Visualisasi Bar Chart
plt.figure(figsize=(10, 6))
mean_metrics.plot(kind='bar')
plt.title("Performa Retrieval per Model")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.savefig(EVAL / "retrieval_barplot.png")
plt.close()

📊 Rata-rata Metrik Retrieval:
        precision@5  recall@5   f1@5  accuracy@5
model                                           
BERT          0.890     0.840  0.860       0.870
SVM           0.710     0.665  0.685       0.675
TF-IDF        0.775     0.610  0.685       0.710


<Figure size 1000x600 with 0 Axes>

In [11]:
# Evaluasi Prediksi Solusi
df_pred = pd.read_csv(prediction_fp)

# Menilai berdasarkan kemiripan string sederhana:
def match(a, b):
    return int(str(a).strip() == str(b).strip())

# Menambahkan ground-truth di sini (simulasi)
manual_ground_truth = {
    1: "Dipidana 2 tahun",
    2: "Mengembalikan uang koperasi",
    3: "Dijatuhi pidana penjara 5 tahun",
    4: "Gugatan tidak diterima",
    5: "Dihukum 1 tahun 6 bulan"
}
df_pred["true_solution"] = df_pred["query_id"].map(manual_ground_truth)

# Hitung akurasi sederhana (string match)
df_pred["correct"] = df_pred.apply(lambda x: match(x["predicted_solution"], x["true_solution"]), axis=1)
df_score = df_pred.groupby("method")["correct"].agg(["sum", "count"])
df_score["accuracy"] = (df_score["sum"] / df_score["count"]).round(3)
df_score = df_score[["accuracy"]]

# Simpan
df_score.to_csv(EVAL / "prediction_metrics.csv")
print("\n✅ Akurasi Prediksi Solusi:")
print(df_score)



✅ Akurasi Prediksi Solusi:
        accuracy
method          
BERT         1.0
SVM          1.0
TF-IDF       1.0
